In [6]:
# encoding: utf-8
import codecs
from bs4 import BeautifulSoup
import re
import sys
import time
import pandas as pd
from unidecode import unidecode
from nltk import sent_tokenize

sys.path.insert(0, '../../base_codes')
sys.path.insert(0, '../../data_specific_codes')
sys.path.insert(0, '../../utility_codes')

from RE_init import *
from main_functions import *
from utility_functions import *
from SDDb_utility_functions import *

stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

%reload_ext autoreload
%autoreload 2


based_dir = ''
file_input_name = 'Sleep_and_Dream_Database_All_Results.html'
file_input = based_dir + file_input_name    

f=codecs.open(file_input, 'r', encoding='utf-8')#'iso-8859-1')#'utf-8')
html = f.read()
start_time = time.time()
parsed_html = BeautifulSoup(html)
print "Execution Time: ", time.time()-start_time, " seconds."
start_time = time.time()
count = 0
list_final_res = []
html_search_results = parsed_html.body.find('ul', attrs={'class':'list_result word-search'})
for ind, item in enumerate(html_search_results.find_all('li')):
    dream_text = item.find('div', attrs={'class':'searchhittext'}).text.strip(' \t\n\r').decode('utf-8')
    dream_text = re.sub(u"(\u2018|\u2019|\u201c|\u201d|\u0060|\u00b4)", "'", dream_text)
    dream_text = dream_text.encode('ascii', 'ignore')
    dream_title = item.find('strong').text
    dream_header_text = item.find('div', attrs={'class':'searchhitheader'}).text
    dream_header_text = dream_header_text.decode('utf-8').encode('ascii', 'ignore')
    dream_date = re.findall(r'\[(.*?)\]', dream_header_text)[0]
    dream_person_id = item.find('a', attrs={'class':'list_result_anchor'}).text
    list_final_res.append([ind, dream_person_id, dream_date, dream_title, dream_text])
    count += 1
print count
print "Execution Time: ", time.time()-start_time
df_texts = pd.DataFrame(list_final_res, columns = ["index", "person id", "answer date", "title", "text"])
print "total number of raw dream reports: ", len(df_texts)

Execution Time:  89.0860738754  seconds.
40435
Execution Time:  22.6938500404
total number of raw dream reports:  40435


In [2]:
df_texts.drop_duplicates(subset=["text"], inplace=True)
start_time = time.time()
for ind, row in df_texts.iterrows():
    post = row["text"]
    num_sentences = len(sent_tokenize(post))    
    post_clean = clean_sent(post)
    if num_sentences == 1 and has_dot_issue(post_clean):
        post_clean = fix_dot_issue(post_clean)
    df_texts.set_value(ind,'text',post_clean)
print "Execution Time: ", time.time()-start_time, " seconds."    
df_texts.to_csv(based_dir + 'SDDb_all_clean.csv',  index = False, encoding='ascii')
print "total number of dream reports after removing duplicates: ", len(df_texts)

Execution Time:  107.876610041  seconds.
total number of dream reports after removing duplicates:  33453


# Start here if you want to load relation

In [1]:
import codecs
from bs4 import BeautifulSoup
import re
import sys
import time
import pandas as pd
from unidecode import unidecode
from nltk import sent_tokenize

sys.path.insert(0, '../../base_codes')
sys.path.insert(0, '../../data_specific_codes')
sys.path.insert(0, '../../utility_codes')

from RE_init import *
from main_functions import *
from utility_functions import *
from SDDb_utility_functions import *

stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

%reload_ext autoreload
%autoreload 2


based_dir = ''
file_input_name = 'SDDb_all_clean.csv'
file_input = based_dir + file_input_name  

def read_df_rel(based_dir, file_input_name):
    file_input = based_dir + file_input_name    
    delim=","
    df = pd.read_csv(file_input,delimiter=delim,header=0)        
    return df

df_all_dreams = read_df_rel(based_dir, file_input_name)

In [ ]:

import time
MAX_ITERATION = 2
annotator = Annotator()
for ind, row in df_rels.iterrows():
    print "ind", ind
    if MAX_ITERATION >= 0:
        if ind > MAX_ITERATION:
            break
    
    start_time = time.time()
    t_annotated = annotator.getAnnotations(row["sentence"], dep_parse=True)
    end_time = time.time()
    print "annotation time: %d (seconds)", (end_time-start_time)
    #t_annotated = row["annotation"]
    #t_annotated = ast.literal_eval(t_annotated)
    
    start_time = time.time()
    g_dir = create_dep_graph(t_annotated)
    end_time = time.time()
    print "creating dependency graph: %d (seconds)", (end_time-start_time)
    
    start_time = time.time()
    rels_pure, rels_simp = get_relations(g_dir, t_annotated,EXTRACT_NESTED_PREPOSITIONS_RELS=True, option="SVO")
    end_time = time.time()    
    print "extracting relationships: %d (seconds)", (end_time-start_time)

In [3]:
from collections import defaultdict
def get_frequent_patterns(g_dir, annotation):
    relations_patterns = defaultdict(list)
    t_verbs = annotation['verbs']
    for v in t_verbs:
        v_id = word_to_node_id(v,annotation)
        try:
            g_dir_v = g_dir[v_id] #adjacency of v_id
        except:
            ##print v_id, " does not appeared as a separate node in parsing tree."
            continue
        nsubj_list = []
        otherNouns_list = []
        for word_id, e in g_dir_v.iteritems():
            if e["rel"] == "nsubj":
                nsubj_list.append(word_id)
            if e["rel"] == "dobj":
                dobj_list.append(word_id)
        if len(nsubj_list) > 0 and len(dobj_list) > 0:
            for s in nsubj_list:
                for o in dobj_list:
                    rel = {}
                    rel["rel"] = v_id
                    rel["arg1"] = s#s.split("-")[0]
                    rel["arg2"] = o#o.split("-")[0]
                    rel["type"] = option
                    rel["pattern"] = "(nsubj, verb, dobj)"
                    rel_expanded = expand_rel(rel.copy(), g_dir, EXTRACT_NESTED_PREPOSITIONS_RELS)
                    #print rel_expanded
                    relations.append(rel_expanded.copy())
                    rel_simp = get_simp_rel(rel_expanded.copy(),option)
                    relations_simp.append(rel_simp)

                    list_conj_and_rels, list_conj_and_rels_simp = get_conj_and_rels(rel.copy(), 
                                                                                    rel_expanded.copy(), g_dir, annotation)

                    relations += list_conj_and_rels
                    relations_simp += list_conj_and_rels_simp

                    list_svp_rels, list_svp_rels_simp = get_svp_rels(rel_expanded.copy())

                    relations += list_svp_rels
                    relations_simp += list_svp_rels_simp
                        
    return relations, relations_simp

# Generating new reports with given keywords 

In [18]:
from nltk.stem import PorterStemmer, WordNetLemmatizer,SnowballStemmer
from nltk.corpus import wordnet as wordnet 
from itertools import chain, repeat
synonyms = wordnet.synsets('give')
set_words=set(chain.from_iterable([word.lemma_names() for word in synonyms]))
new_set=[]
for word in set_words:
    word=word.replace('_',' ')
    new_set.append(word)
    print word
print new_set
# co refrence resolution

feed
springiness
render
give
spring
contribute
move over
kick in
consecrate
have
pass
apply
establish
founder
open
grant
pay
make
devote
throw
cave in
impart
fall in
chip in
return
collapse
turn over
afford
sacrifice
give way
reach
hand
break
hold
generate
present
gift
dedicate
yield
leave
ease up
commit
pass on
[u'feed', u'springiness', u'render', u'give', u'spring', u'contribute', u'move over', u'kick in', u'consecrate', u'have', u'pass', u'apply', u'establish', u'founder', u'open', u'grant', u'pay', u'make', u'devote', u'throw', u'cave in', u'impart', u'fall in', u'chip in', u'return', u'collapse', u'turn over', u'afford', u'sacrifice', u'give way', u'reach', u'hand', u'break', u'hold', u'generate', u'present', u'gift', u'dedicate', u'yield', u'leave', u'ease up', u'commit', u'pass on']


In [12]:
df_new_reports = df_texts[df_texts.apply(lambda x: is_any_entities_present(x["text"], new_set), axis=1)]
print "number of new reports:", len(df_new_reports)
# df_car.to_csv(based_dir + 'SDDb_car.csv',  index = False, encoding='ascii')


number of new reports: 706


# Generating entities
New Idea: why not measure each vector similarity between two entity and then cluster them in one group

In [31]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
from textblob import TextBlob

def get_continuous_chunks(text):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    prev = None
    continuous_chunk = []
    current_chunk = []
    for i in chunked:
        if type(i) == Tree:
                current_chunk.append(" ".join([token for token, pos in i.leaves()]))
        elif current_chunk:
                named_entity = " ".join(current_chunk)
                if named_entity not in continuous_chunk:
                        continuous_chunk.append(named_entity)
                        current_chunk = []
        else:
            continue
    return continuous_chunk
 
my_sent = "Nightmare in Cambodia. In the dream we are being overrun by sappers who have got past the Night Defensive Perimeter trips and claymores and now crawl forward. I wake up and see a boot tread close to my face. I slowly withdraw my . 45 from its holster, pull the hammer back, then aim it at the boot. Just then the cloud. obscured moon comes out and I realize the boot is American and that it is Jerry Bieck's foot. In the pitch stillness I point the . 45 straight up in the air. Pinching the hammer tightly I pull the trigger and settle the hammer back in place. I re. holster the pistol and go back to sleep. The next day, after a very difficult march, all the men are overjoyed to be out of Cambodia. I tell no one what almost happened."
print get_continuous_chunks(my_sent)
blob = TextBlob(my_sent)
print blob.noun_phrases

['Nightmare', 'Cambodia', 'Night', 'American', 'Jerry Bieck']
['nightmare', 'cambodia', u'defensive perimeter', u'boot tread', u'jerry bieck', u"'s foot", u'pitch stillness', 'pinching', u'difficult march', 'cambodia']


In [ ]:
# let's take one of dreams and try to find relations
i=0
for ind, row in df_texts.iterrows():
    if(i==0):
        txt=row['text']
        print row['text']
    break

In [47]:
df_rels

,A2,AM-DIS,AM-LOC,AM-MNR,AM-NEG,AM-TMP,C-V,R-A0,arg1,arg1_prepositions,arg1_with_pos,arg2,arg2_prepositions,arg2_with_pos,pattern,rel,rel_prepositions,rel_with_pos,type
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{we},,we-PRP-1,in a {room},REASON: prep_in TEXT: {were} --,a-DT-5 room-NN-6,"(nsubj, verb (no obj), prep)",{were} alone,REASON: prep_in TEXT: a {room} --,were-VBD-2 alone-RB-3,SVP
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{he},,he-PRP-8,a {date},REASON: prep_with TEXT: a Chinese {girl} --,a-DT-10 date-NN-11,"(nsubj, verb, dobj)",{had},,had-VBD-9,SVO
2,NaN,NaN,NaN,NaN,NaN,later that night,NaN,NaN,{he},NaN,NaN,{a date with a Chinese girl},NaN,NaN,"(srl-A0, srl-v, srl-A1)",{had},NaN,NaN,SRL
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{him},,him-PRP-4,with {me},REASON: prep_with TEXT: {sleep} --,me-PRP-8,"(nsubj, verb (no obj), prep)",to {sleep},REASON: prep_with TEXT: {me} --,to-TO-5 sleep-VB-6,SVP
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{I},NaN,NaN,{him to sleep with me and took my shirt off},NaN,NaN,"(srl-A0, srl-v, srl-A1)",{expected},NaN,NaN,SRL
5,NaN,NaN,NaN,NaN,NaN,NaN,off,NaN,{him},NaN,NaN,{my shirt},NaN,NaN,"(srl-A0, srl-v, srl-A1)",{took},NaN,NaN,SRL
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{The top part of my body , naked ,}",NaN,NaN,{beautiful},NaN,NaN,"(srl-A0, srl-v, srl-A1)",{looked},NaN,NaN,SRL
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{him},,him-PRP-3,{me},,me-PRP-6,"(nsubj, verb, dobj)",to {caress},,to-TO-4 caress-VB-5,SVO
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{I},NaN,NaN,{him to caress me but he was planning to go},NaN,NaN,"(srl-A0, srl-v, srl-A1)",{expected},NaN,NaN,SRL
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{he},NaN,NaN,{to go},NaN,NaN,"(srl-A0, srl-v, srl-A1)",{planning},NaN,NaN,SRL


In [48]:
df_output

,A2,A3,A4,AM-ADV,AM-CAU,AM-DIR,AM-DIS,AM-LOC,AM-MNR,AM-MOD,...,arg2_prepositions,arg2_with_pos,pattern,post_num,rel,rel_prepositions,rel_with_pos,sentence,sentence_num,type
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"(srl-A0, srl-v, srl-A1)",1,{overrun},NaN,NaN,In the dream we are being overrun by sappers w...,1,SRL
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"(srl-A0, srl-v, srl-A1)",1,{got},NaN,NaN,In the dream we are being overrun by sappers w...,1,SRL
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,REASON: prep_close_to TEXT: {tread} --,my-PRP$-11 face-NN-12,"(nsubj, verb (no obj), prep)",2,{tread},REASON: prep_close_to TEXT: my {face} --,tread-VB-8,I wake up and see a boot tread close to my face.,2,SVP
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"(srl-A0, srl-v, srl-A1)",2,{see},NaN,NaN,I wake up and see a boot tread close to my face.,2,SRL
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,my-PRP$-4,"(nsubj, verb, dobj)",3,slowly {withdraw},,slowly-RB-2 withdraw-VB-3,I slowly withdraw my .,3,SVO
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,slowly,NaN,...,NaN,NaN,"(srl-A0, srl-v, srl-A1)",3,{withdraw},NaN,NaN,I slowly withdraw my .,3,SRL
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,the-DT-7 hammer-NN-8,"(nsubj, verb, dobj)",4,{pull} back,REASON: prep_from TEXT: its {holster} --,pull-VB-6 back-RB-9,"45 from its holster, pull the hammer back, the...",4,SVO
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,"(nsubj, verb, (O)prep)",4,{pull} back <<the {hammer}>> from,,pull-VB-6 back-RB-9,"45 from its holster, pull the hammer back, the...",4,SV(O)P
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,at the boot,NaN,NaN,...,NaN,NaN,"(srl-A0, srl-v, srl-A1)",4,{aim},NaN,NaN,"45 from its holster, pull the hammer back, the...",4,SRL
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"(srl-A0, srl-v, srl-A1)",6,{realize},NaN,NaN,obscured moon comes out and I realize the boot...,6,SRL


In [13]:
%matplotlib inline

'''
PARAMETERS
'''
SEPARATE_SENT = True 
SHOW_DP_PLOTS = False
SHOW_REL_EXTRACTIONS = True
NODE_SELECTION = False
MAX_ITERATION = -1 #-1 -> to try all
SAVE_GEFX = True
SAVE_G_JSON = True
SAVE_PAIRWISE_RELS = True
SAVE_ALL_RELS = False
CLEAN_SENTENCES = True
SET_INOUT_LOC_FROM_PYTHON_ARGS = False 
SHOW_ARGUMENT_GRAPH = False
EXTRACT_NESTED_PREPOSITIONS_RELS = True 
# DATA_SET = "fakenews"
INPUT_DELIMITER = "\n"#","#"\n"
SAVE_ANNOTATIONS_TO_FILE = True
LOAD_ANNOTATIONS = False#True #False 
KEEP_ORDER_OF_EXTRACTIONS = True 
PRINT_EXCEPTION_ERRORS = True #U still need to uncomment some of the error messages if u want to see all the exception errors.
SAVE_ALL_SENTENCES_AND_ANNOTATIONS = True


from nltk.tokenize import sent_tokenize
from practnlptools.tools import Annotator
annotator = Annotator()
output = []
for i, row in df_new_reports.iterrows():
    start_time = time.time()
    all_rels_str, all_rels = [], []
    t_orig = row['text']
    t_orig = t_orig.replace("(","")
    t_orig = t_orig.replace(")","")
    t_orig = t_orig.replace("\n",".")
    t_orig = clean_sent(t_orig)
    t_sentences = sent_tokenize(t_orig)
    for t_ind, t in enumerate(t_sentences):
        t_annotated = annotator.getAnnotations(t, dep_parse=True)
        dep = t_annotated['dep_parse']
        g_dir = create_dep_graph(t_annotated)
        if g_dir is None:
            print "No extraction found"
            continue
        rels_pure, rels_simp = get_relations(g_dir, t_annotated,EXTRACT_NESTED_PREPOSITIONS_RELS=True, option="SVO")
        rels = rels_pure
        all_rels_str = all_rels_str + get_rels_str(rels) #For simply counting the exact strings
        all_rels = all_rels + rels # to later create a dataframe
        for r in rels:
            output_row = defaultdict(list)
            output_row = r.copy()
            #output_row["original_text"] = t_orig
            output_row["sentence"] = t
            if SAVE_ANNOTATIONS_TO_FILE:
                output_row["annotation"] = t_annotated
            
            if KEEP_ORDER_OF_EXTRACTIONS:
                #if "post_num" in df.columns:
                #    post_num_tmp = df.iloc[ind]["post_num"]
                #else:
                post_num_tmp = t_ind
                #if "sentence_num" in df.columns:
                #    sentence_num_tmp = df.iloc[ind]["sentence_num"]
                #else:
                sentence_num_tmp = t_ind                    
                output_row["post_num"] = post_num_tmp
                output_row["sentence_num"] = sentence_num_tmp
            output.append(output_row) 
            
            #dict_writer.writerow(output_row)
                
                  
    df_rels = pd.DataFrame(all_rels)
    df_output = pd.DataFrame(output)

    #print df_rels
    #'''
#     DATA_SET = "fakenews"
#     input_name_prefix = buzzfeed_fake_files[i]
#     start_time = time.time()
#     rels_to_network(df_rels,
#                     input_fname=input_name_prefix,
#                     output_dir_arg=output_dir,
#                     MAX_ITERATION=-1,
#                     NODE_SELECTION=False,
#                     DATA_SET=DATA_SET,
#                     SAVE_GEFX=False,
#                     SAVE_PAIRWISE_RELS=True,
#                     SHOW_ARGUMENT_GRAPH=True,
#                     SAVE_G_JSON=False,
#                     SAVE_DF_SELECTED=False,
#                     PATH_TO_SAVED_ARG_GRAPH=output_dir+input_name_prefix+".svg")

    end_time = time.time()
    print "execution time:", (end_time - start_time)/60.0 , " min."

execution time: 0.482268265883  min.
No extraction found
No extraction found
execution time: 0.157395931085  min.
execution time: 0.574659649531  min.
execution time: 0.483790584405  min.
execution time: 0.312998048464  min.
execution time: 0.603793549538  min.
execution time: 0.464242200057  min.
execution time: 0.126364966234  min.
execution time: 0.721887036165  min.
execution time: 0.0958084980647  min.
execution time: 0.245797614257  min.
execution time: 0.560326000055  min.
execution time: 0.148320047061  min.
execution time: 0.274940868219  min.
Tokenizer failed during parsing, Ex. there might be a dash in the sentence!
No extraction found
execution time: 0.111824746927  min.
execution time: 0.0401945312818  min.
execution time: 0.175248281161  min.
execution time: 0.542168569565  min.
execution time: 0.111369935671  min.
execution time: 0.171329383055  min.
execution time: 0.195214815934  min.
execution time: 0.405212116241  min.
execution time: 0.0414530158043  min.
execution 

KeyboardInterrupt: 

In [16]:
from nltk.corpus import stopwords
stopwords.words('english')
stemmer2 = SnowballStemmer("english")
for word in get_top_entities(df_output,top_num=100)['entity']:
    if stemmer2.stem(word)  not in stopwords.words('english'):
        print word

[nltk_data] Downloading package stopwords to /Users/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{I}
{it}
{she}
{me}
{we}
{he}
{they}
{her}
{you}
{something}
{this}
{that}
{water}
{one}
{It}
{We}
{him}
{dream}
{what}
{mother}
{them}
{time}
{kind}
{woman}
{me.}
{Mom}
{people}
{baby}
{She}
{thing}
{cake}
{all}
{myself}
{He}
{someone}
{it.}
{reason}
{man}
{Lila}
{car}
{They}
{us}
{part}
{pool}
{girl}
{way}
{i}
{head}
{person}
{guy}
{music}
{house}
{sort}
{Kees}
{Marty}
{life}
{aware}
{which}
{afraid}
{place}
{point}
{hair}
{body}
{room}
{blade}
{Dad}
{some}
{trombone}
{sure}
{things}
{crack}
{lot}
{himher}
{name}
{home}
{bleachers}
{times}
{guys}
{boat}
{song}
{who}
{father}
{elevator}
{bus}
{him or her}
{hand}
{My mother}
{able}
{rabbit}
{e.}
{snake}
{side}
{him.}
{there}
{effects}
{One}
{bottom}
{abortion}
{more}
{bed}


In [91]:
from nltk.corpus import stopwords
stopwords.words('english')
for word in get_top_entities(df_output,top_num=100)['entity']:
    if word  not in stopwords.words('english'):
        print word
# print_full(get_top_entities(df_output,top_num=100))

{I}
{it}
{we}
{me}
{she}
{he}
{they}
{We}
{you}
{her}
{It}
{him}
{water}
{something}
{that}
{this}
{She}
{them}
{He}
{one}
{dream}
{card}
{it.}
{me.}
{They}
{people}
{what}
{Mom}
{kind}
{house}
{us}
{time}
{man}
{pool}
{mother}
{myself}
{point}
{someone}
{car}
{swimming}
{way}
{place}
{room}
{area}
{beach}
{thing}
{woman}
{Dad}
{girl}
{part}
{guy}
{all}
{head}
{boat}
{him.}
{sort}
{Lila}
{back}
{sure}
{i}
{wave}
{reason}
{baby}
{things}
{was}
{everyone}
{some}
{kids}
{body}
{bus}
{school}
{hand}
{name}
{afraid}
{father}
{home}
{bed}
{anything}
{cake}
{which}
{e.}
{us.}
{lot}
{who}
{door}
{person}
{life}
{there}
{hair}
{able}
{aware}
{end}
{song}
{husband}
{them.}
{stuff}
{'I}
{side}
{times}
{Kees}


In [41]:
from itertools import chain
from nltk.corpus import wordnet

# synonyms = wordnet.synsets(text)
# lemmas = set(chain.from_iterable([word.lemma_names() for word in synonyms]))
synonyms = wordnet.synsets('change')
set(chain.from_iterable([word.lemma_names() for word in synonyms]))

{u'alter',
 u'alteration',
 u'change',
 u'commute',
 u'convert',
 u'deepen',
 u'exchange',
 u'interchange',
 u'modification',
 u'modify',
 u'shift',
 u'switch',
 u'transfer',
 u'variety',
 u'vary'}

In [43]:
dog = wordnet.synset('dog.n.01')
print dog.lemma_names

<bound method Synset.lemma_names of Synset('dog.n.01')>


In [45]:
# Dictionary: given a word, what are the different meaning of the word

for i,j in enumerate(wordnet.synsets('dog')):
    print "Meaning",i, "NLTK ID:", j.name
    print "Definition:",j.definition

Meaning 0 NLTK ID: <bound method Synset.name of Synset('dog.n.01')>
Definition: <bound method Synset.definition of Synset('dog.n.01')>
Meaning 1 NLTK ID: <bound method Synset.name of Synset('frump.n.01')>
Definition: <bound method Synset.definition of Synset('frump.n.01')>
Meaning 2 NLTK ID: <bound method Synset.name of Synset('dog.n.03')>
Definition: <bound method Synset.definition of Synset('dog.n.03')>
Meaning 3 NLTK ID: <bound method Synset.name of Synset('cad.n.01')>
Definition: <bound method Synset.definition of Synset('cad.n.01')>
Meaning 4 NLTK ID: <bound method Synset.name of Synset('frank.n.02')>
Definition: <bound method Synset.definition of Synset('frank.n.02')>
Meaning 5 NLTK ID: <bound method Synset.name of Synset('pawl.n.01')>
Definition: <bound method Synset.definition of Synset('pawl.n.01')>
Meaning 6 NLTK ID: <bound method Synset.name of Synset('andiron.n.01')>
Definition: <bound method Synset.definition of Synset('andiron.n.01')>
Meaning 7 NLTK ID: <bound method Syns

In [17]:
get_top_extractions(df_output)

,relation tuple,counts
0,{I};{had};a {dream},11
1,{I};{say};{something},6
2,{I};{find};{him or her},6
3,{I};{tell};{her},4
4,{I};{love};{you},4
5,{I};{had};{a dream},4
6,{I};already {have};{some},4
7,{he};{said};{something},4
8,{I};{get};{it},4
9,{this};how will {help};{her},4
